In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rc
from scipy.signal import find_peaks 
from cProfile import label

In [1]:
rho_HCl = 1180 #g/L
conc = 0.37 #%
M_w = 36.46 #g/mol
c1 = 0.1 #mol/L
V1 = 0.1 #L
c_hcl = rho_HCl/M_w*conc
V2 = (c1*V1)/c_hcl
print(c_hcl, V2, V2*1000)
V3 = (c1*0.25)/c_hcl
print(V3*1000)

e_A = 0.0001
e_WL = 0.1
e_V = 0.04/1000
e_m = 0.001

11.974766867800328 0.0008350893266147506 0.8350893266147505
2.0877233165368763


In [53]:
MnCl2_01M_HCl_path = "./MnCl2_in_HCl.CSV"
MnCl2_hcl=pd.read_csv(MnCl2_01M_HCl_path, sep=';', decimal=',', index_col = 0)
# display(MnCl2_hcl)

In [ ]:
V_array = np.full(7, 0.01)
m_array = np.array([6.096, 3.300, 1.644, 0.853, 0.436, 0.229, 0.114])
WL_full = np.array([202, 309.2, 335.8, 355.6, 400.2, 432.0, 525.4, 
                    235, 313.8, 339.4, 358.4, 406.0, 432.6, 529.2]).reshape(2,7)
A_peak = np.array([1.6172, 0.0169, 0.0211, 0.0196, 0.0186, 0.009, 0.0098,  
                    2.2859, 0.0882, 0.1032, 0.1618, 0.1026, 0.0632, 0.0676]).reshape(2,7)

In [100]:

def conc_MnCl2(m, V):
  c = m/(197.91*V)
  return c

def epsilon(A, c):
  eps = A/c
  return eps

def H_conc(pH, c, V):
  c_H = 10**(-pH)
  V_H = (c*V)/c_H
  return c_H, V_H

def energy(wl):
  E = ((6.626*10**(-34))*(2.998*10**(10)))/wl
  eV = E*6.2415*10**(18)
  return E, eV

def eprop(c, e_m, m, e_V, V, eps, e_A, A):
  e_conc = c*np.sqrt(((e_m/m)**(2))+((e_V/V)**(2)))
  e_eps = eps*np.sqrt((e_A/A)**(2)+(e_conc/c)**(2))
  return e_conc, e_eps

def E_range(E_high, E_low):
  E_mid = (E_high + E_low)/2
  E_un = (E_high - E_low)/2
  return E_mid, E_un

In [102]:
c_list = conc_MnCl2(m_array, V_array)
print(c_list)

[3.08018796 1.66742459 0.83068061 0.43100399 0.22030216 0.11570916
 0.05760194]


In [103]:

E_j, E_eV = energy(WL_full)

E_mid, E_un= E_range(E_j[0], E_j[1])
eV_mid, eV_un = E_range(E_eV[0], E_eV[1])
print(E_mid)
print(E_un)
print(eV_mid) 
print(eV_un)

[9.14355896e-26 6.37747385e-26 5.88427510e-26 5.56444076e-26
 4.92825011e-26 4.59513245e-26 3.76730626e-26]
[6.90474704e-27 4.70888920e-28 3.13735047e-28 2.18213363e-28
 3.54550368e-28 3.18884972e-29 1.35745911e-28]
[5.70695232e-07 3.98050030e-07 3.67267030e-07 3.47304570e-07
 3.07596731e-07 2.86805192e-07 2.35136420e-07]
[4.30959787e-08 2.93905319e-09 1.95817729e-09 1.36197871e-09
 2.21292612e-09 1.99032055e-10 8.47258105e-10]


In [104]:

eps = epsilon(A_peak, c_list)
print(eps)

[[0.52503289 0.01013539 0.02540086 0.04547522 0.0844295  0.07778122
  0.17013316]
 [0.74213003 0.05289595 0.12423547 0.37540256 0.46572399 0.54619703
  1.17357158]]


In [105]:

error_conc, error_eps = eprop(c_list, e_m, m_array, e_V, V_array, eps, e_A, A_peak)
print(error_conc)
print(error_eps)

[0.01233111 0.00668881 0.00336092 0.00179654 0.00101579 0.00068522
 0.00055533]
[[2.10214761e-03 7.24553619e-05 1.58284861e-04 2.99602605e-04
  5.97994368e-04 9.79320389e-04 2.38835709e-03]
 [2.97119279e-03 2.20502508e-04 5.16869466e-04 1.58188181e-03
  2.19486284e-03 3.34799864e-03 1.14466884e-02]]


In [34]:
m = 10**(-5)*197.91*0.01
V_1 = (0.01*10**(-5))/0.05760194
print(m, V_1)
print(V_1/2, V_1/4)

1.9791e-05 1.736052639893726e-06
8.68026319946863e-07 4.340131599734315e-07
